In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Ресайз обработанных изображений до разрешения 1024*1024

In [ ]:
import os
import cv2

images_names = os.listdir('/your_deoldify_images_path/') 
for name in images_names:
    try:
        image = cv2.imread(f'/your_deoldify_images_path/{name}')
        resized = cv2.resize(image, (1024,1024)) 
        cv2.imwrite(f'/your_deoldify_images1024_path/{name}', resized) 
    except Exception:
        pass


#Устанавливаем необходимые зависимости и клонируем проект с github

In [ ]:
!pip install pyspng ninja imageio-ffmpeg==0.4.3
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch
!pip install gdown 
%cd ./stylegan2-ada-pytorch

#Подготовка данных перед обучением
###Создаем zip архив с изображениями и их метаинформацией в формате json. Здесь необходимо указать путь к обработанным и ресайзнутым картинкам, так же указать путь к архиву, в котором будут хранится все тренировочные данные(архив создается автоматически)

In [ ]:
!python dataset_tool.py --source=/your_deoldify_images1024_path/ --dest=/your_path/pics.zip

#Тренировка модели 
###Здесь необходимо указать директорию в которой будет сохранятся результат обучения и указать подготовленный тренировочный архив pics.zip. Подробнее о передаваемых аргументах: https://github.com/NVlabs/stylegan2-ada-pytorch

In [ ]:
!python train.py \
--outdir=/your_out_dir/ \
--data=/your_path/pics.zip \
--snap=10 \
--mirror=1 \
--gpus=1 \
--aug=ada \
--target=0.7

#Генерация изображений на выбранной сети

In [ ]:
!python generate.py \
--outdir=/your_out_dir/ \
--trunc=0.5 \
--seeds=50-100 \
--network=/your_network_path/

#Постобработка изображений
###Так как для тренировки сети необходимы квадратные изображения для лучшей сходимости и разрешение должно быть степенью 2, вначале мы изменили разрешение на 1024х1024, в таком разрешении оригинальное изображение сжато по ширине, его необходимо восстановить до исходных размеров 1280х960

In [ ]:
import os
import cv2

images_names = os.listdir('/your_generated_images_path/') 
for name in images_names:
    try:
        image = cv2.imread(f'/your_generated_images_path/{name}')
        resized = cv2.resize(image, (1280,960)) 
        cv2.imwrite(f'/your_final_images_path/{name}', resized) 
    except Exception:
        pass
